### requirement
konlpy, pytorch-crf, transformers

In [ ]:
!pip install konlpy
!pip install pytorch-crf
!pip install transformers

In [61]:
from transformers import *
import numpy as np
import torch
from torch import nn
import pandas as pd
from torchcrf import CRF
from collections import defaultdict
import logging
import os
import unicodedata
from shutil import copyfile
from transformers import PreTrainedTokenizer
from konlpy.tag import Okt

# ● DistilKoBERT 모델 가져오기
* train.ipython에서 훈련된 DistilBERT + CRF 모델과 구조를 가져온다.

In [133]:
logger = logging.getLogger(__name__)

class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = {"vocab_file": "tokenizer_78b3253a26.model","vocab_txt": "vocab.txt"}
    pretrained_vocab_files_map = {
        "vocab_file": { "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model"},
        "vocab_txt": {"monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt"} }
    max_model_input_sizes = {"monologg/kobert": 512}
    pretrained_init_configuration = {"monologg/kobert": {"do_lower_case": False}}

    def __init__(self,vocab_file,vocab_txt,do_lower_case=False,remove_space=True,keep_accents=False,unk_token="[UNK]",sep_token="[SEP]",pad_token="[PAD]",cls_token="[CLS]",mask_token="[MASK]",**kwargs):
        super().__init__(unk_token=unk_token,sep_token=sep_token,pad_token=pad_token,cls_token=cls_token,mask_token=mask_token,**kwargs)

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [134]:
class DistilKobertCRF(nn.Module):
    def __init__(self, num_classes):
        super(DistilKobertCRF, self).__init__()

        self.hidden_size = 768
        self.num_classes = num_classes
        self.pad_id = 1

        self.bert = DistilBertModel.from_pretrained("monologg/distilkobert")
        self.FC = torch.nn.Linear(self.hidden_size,self.num_classes)
        self.crf = CRF(num_tags = num_classes, batch_first = True)

    def forward(self, input_ids, real_tags = None):
        attention_mask = input_ids.ne(self.pad_id).float()
        last_hidden_state = self.bert.forward(input_ids, attention_mask)
        dense = self.FC(last_hidden_state[0])
        
        if real_tags is not None:
            log_likelihood = self.crf(dense,real_tags)
            pred_tags = self.crf.decode(dense)
            return log_likelihood, pred_tags
        
        else:
            pred_tags =  self.crf.decode(dense)
            return pred_tags

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


# ● Example
* EBS 최태성의 한국사 강의 중 4강을 이용하여 키워드 추출 결과를 확인

In [169]:
context = pd.read_table("/content/drive/My Drive/projects/04강 고려의 발전과 변화.txt", header=None)
context

,0
0,"안녕하세요, 여러분. 역사는 최태성! 빵! 지금 수능특강을 열심히 열심히 달리고 있..."
1,우리가 드디어 중세 와우! 70만 년 역사 끝냈고 천 년의 고대 끝냈고 이제 중세 ...
2,고려 오백 년이고요. 그다음이 조선 오백 년. 이렇게 합쳐서 천 년이 있어요.
3,"그중에서 중세 고려 오백여 년의, 그 오백 년 조금 안 되겠구나. 역사를 한번 살펴..."
4,앞에서 한번 설명드렸지만 고려는 호족의 시대입니다. 호족이 나라를 세웠죠. 대표적인...
...,...
249,"호족의 시대, 문벌귀족의 시대, 무신의 시대, 권문세족의 시대, 신진사대부의 시대...."
250,"외침에 맞서 싸웠던 거란, 여진, 몽골, 홍건적과 왜구에 맞서 싸웠던 그런 역사. ..."
251,"비록 충 자가 들어가고 부마국이 되었지만, 그래도 우리의 자주성을 잃지는 않았다는 사실."
252,이걸 여러분들이 고려인들을 통해서 배우면 어떨까 하는 생각이 듭니다. 그런 DNA를...


# ● 1 ) NER inference

In [135]:
nr_model = DistilKobertCRF(num_classes=30)
nr_model.load_state_dict(torch.load("/content/drive/My Drive/projects/ner_model_weight_concat"))
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
max_len = 85

In [136]:
def ner_inference(test_sentence):
    label_to_tag = {0: 'PER_B', 1: 'DAT_B',2: '-',3: 'ORG_B',4: 'CVL_B',5: 'NUM_B',6: 'LOC_B',
                    7: 'EVT_B',8: 'TRM_B',9: 'TRM_I',10: 'EVT_I',11: 'PER_I',12: 'CVL_I',
                    13: 'NUM_I',14: 'TIM_B', 15: 'TIM_I',16: 'ANM_B', 17: 'DAT_I',
                    18: 'FLD_B',19: 'ORG_I', 20: 'MAT_B',21: 'MAT_I', 22: 'AFW_B', 23: 'LOC_I',
                    24: 'AFW_I',25: 'PLT_B',26: 'FLD_I',27: 'ANM_I',28: 'PLT_I', 29: '[PAD]'}

    tokenized_sentence = torch.tensor([tokenizer.encode(test_sentence,truncation=True, max_length=max_len, pad_to_max_length=True)])
    ans = nr_model.forward(tokenized_sentence,real_tags=None)

    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])

    new_tokens, new_labels = [], []
    for token, label_idx in zip(tokens, ans[0]):

        if token.startswith("▁"):
            new_labels.append(label_to_tag[label_idx])
            new_tokens.append(token[1:])
        elif token not in ['[CLS]', '[SEP]', '[PAD]']:
            new_tokens[-1] = new_tokens[-1] + token

    ner_dict = defaultdict(list)
    for token, label in zip(new_tokens, new_labels):
        if label == "-":continue
        ner_dict[label].append(token)
    return ner_dict

In [57]:
for i in range(len(context)):
    print(i, "th", list(ner_inference(context.iloc[i,0]).items()))

0 th [('PER_B', ['최태성!'])]
1 th [('NUM_B', ['70만', '오백']), ('NUM_I', ['년'])]
2 th [('NUM_B', ['천']), ('NUM_I', ['년이'])]
3 th [('DAT_B', ['중세']), ('NUM_B', ['오백여'])]
4 th [('LOC_B', ['고려는'])]
5 th [('LOC_B', ['고려']), ('DAT_B', ['오백']), ('NUM_B', ['두', '1170년이에요.']), ('NUM_I', ['시기를'])]
6 th [('DAT_B', ['1170년이', '1170년은'])]
7 th [('NUM_B', ['6두품이', '6두품이'])]
8 th [('CVL_B', ['문벌귀족화되고요.'])]
9 th [('CVL_B', ['문벌귀족들이'])]
10 th [('LOC_B', ['원나라가', '몽골이'])]
11 th []
12 th []
13 th [('LOC_B', ['향리']), ('CVL_B', ['권문세족을'])]
14 th [('LOC_B', ['조선을'])]
15 th []
16 th [('NUM_B', ['하나씩', '하나씩']), ('LOC_B', ['고려를'])]
17 th [('LOC_B', ['신라'])]
18 th [('PER_B', ['원종과', '애노의'])]
19 th []
20 th [('LOC_B', ['흑창.', '고구려']), ('CVL_B', ['고국천왕'])]
21 th [('DAT_B', ['봄에', '가을에']), ('CVL_B', ['백성들이'])]
22 th []
23 th [('NUM_B', ['두']), ('NUM_I', ['번째'])]
24 th [('CVL_B', ['호족들의', '당근과', '채찍']), ('NUM_B', ['혼자'])]
25 th [('CVL_B', ['호족들에게'])]
26 th []
27 th [('PER_B', ['왕건의']), ('CVL_B', ['부인이']), ('NUM_B', ['

# ● 2 ) TF-IDF

In [161]:
# TF-IDF score 계산용 함수

from math import log10

def f(t, d):
    # d is document == tokens
    return d.count(t)

def tf(t, d):
    # d is document == tokens
    return 0.5 + 0.5*f(t,d)/max([f(w,d) for w in d])

def idf(t, D):
    # D is documents == document list
    numerator = len(D)
    denominator = 1 + len([ True for d in D if t in d])
    return log10(numerator/denominator)

def tfidf(t, d, D):
    return tf(t,d)*idf(t, D)

def tokenizer_for_tfidf(d):
    twit = Okt()
    def keyword_extractor(text):
        tokens = twit.phrases(text)
        tokens = [token for token in tokens if len(token) > 1]
        count_dict = [(token, text.count(token)) for token in tokens]
        ranked_words = sorted(count_dict, key=lambda x: x[1], reverse=True)[:20]
        return [keyword for keyword, freq in ranked_words]
    return keyword_extractor(d)

def tfidfScorer(D):
    tokenized_D = [tokenizer_for_tfidf(d) for d in D]
    result = []
    for d in tokenized_D:
        result.append([(t, tfidf(t, d, tokenized_D)) for t in d])
    return result

In [173]:
for i in range(len(context)):
    extracted_keywords = tfidfScorer([context.iloc[i,0]])[0]
    res_tfidf = [keyword for keyword, score in sorted(extracted_keywords, key=lambda x: x[1], reverse=True)]
    okt = Okt()
    refined_tfidf = [okt.morphs(w)[0] for w in res_tfidf]
    print(i, "th", set(refined_tfidf))

0 th {'달리', '역사', '여러분', '지금', '최태성', '수능특강'}
1 th {'70만', '우리', '역사', '오백', '중세', '이제', '와우', '고대', '천'}
2 th {'이', '오백', '조선', '고려', '그다음'}
3 th {'오백', '역사', '중세', '한번', '고려', '조금', '그중', '그', '시간'}
4 th {'왕건', '설명', '바로', '고려', '한번', '나라', '시대', '호족'}
5 th {'구분', '먼저', '바로', '오백', '시기', '고려', '두', '분기점', '항상', '일단', '시대', '때'}
6 th {'무엇', '전기', '뭔', '1170년', '후기', '기점', '때문', '무신정변'}
7 th {'두품', '설명', '이', '한번', '세운', '나라', '호족', '시간', '전기'}
8 th {'문벌귀족', '이', '세력', '지방', '기득권', '이제', '중앙', '향리'}
9 th {'문벌귀족', '무신정권', '영원한', '설명'}
10 th {'몽골', '이', '무신', '원나라', '그'}
11 th {'성장한', '이', '우리', '이야기', '세력', '그', '권문세족'}
12 th {'세력', '등장', '막바지', '시대', '이제'}
13 th {'공격', '세력', '수용', '성리학', '누구', '신진사대부', '출신', '그', '향리', '권문세족'}
14 th {'구분', '이', '머릿속', '바로', '인지', '시기', '조선', '나라', '여러분'}
15 th {'문벌귀족', '머릿속', '무신', '신진사대부', '기억', '호족', '흐름', '권문세족'}
16 th {'하나', '고려', '왕', '시대', '이제', '호족'}
17 th {'왕건', '설명', '말기', '태조', '나라', '혼란', '신라', '얼마나', '여러', '정치'}
18 th {'원종', '이', '목표', '통제'

# ● Keyword Extraction
* 강의 자막을 한 문장씩 입력받는다.
* NER 결과와 TF-IDF 결과의 교집합을 이용하여 추천한다.
* 위키 백과에서 검색한 결과를 함께 return한다.

In [197]:
import requests, re
from bs4 import BeautifulSoup

def extract_keyword(context): # 한 문장

    # ner inference 결과
    res_ner = ner_inference(context)

    # TF-IDF 결과
    extracted_keywords = tfidfScorer([context])[0]
    res_tfidf = [keyword for keyword, score in sorted(extracted_keywords, key=lambda x: x[1], reverse=True)] # score에 대해 내림차순 정렬한 후 list에 추가

    # 조사 제거
    okt = Okt()

    refined_ner = []
    for tag in ['ORG_B','PER_B','CVL_B','EVT_B','ORG_I','PER_I','PER_I','CVL_I','EVT_I']:
        candidate = res_ner.get(tag)
        if candidate:
            refined_ner.extend([okt.morphs(w)[0] for w in candidate])

    refined_tfidf = [okt.morphs(w)[0] for w in res_tfidf]

    # 동시에 나타난 keyword만 추출
    res = set([w for w in refined_tfidf if len(w) >=2 and w in refined_ner])

    # 위키 스크래핑 후 결과 저장
    search_result = []
    for keyword in res:
        url = f'http://encykorea.aks.ac.kr/Contents/SearchNavi?keyword={keyword}&ridx=0&tot=124'
        resp = requests.get(url)
        html = resp.text
        soup = BeautifulSoup(html, "html.parser")
        soup_find = soup.find("div", class_="tx")
        if soup_find is None :
            continue
        search = soup_find.getText()
        search_result.append(search)

    return dict(zip(res, search_result))

In [191]:
extract_keyword('문벌귀족 이자겸. ‘그래? 그럼 내가 한번 왕 해 볼까?’ 이런 마음이 올 것 아니에요? 결국은 이자겸의 난이 벌어지게 됩니다.')

{'문벌귀족': '대대로 내려오는 그 집안의 사회적 신분이나 지위.',
 '이자겸': '고려전기 상서좌복야, 협모안사공신 수태사 중서령 소성후 등을 역임한 관리.문신.'}

In [192]:
print(context.iloc[179,0],"\n")
extract_keyword(context.iloc[179,0])

공민왕의 어떤 반원 자주 정책. 이걸 다 부정합니다. 쌍성총관부 없애고요. 정동행성 이문소 없애고요. 정방 없애고요. 



{'공민왕': '고려후기 제31대(재위: 1351~1374) 왕.',
 '쌍성총관부': '고려 후기 몽고가 고려의 화주(和州) 이북을 직접 통치하기 위해 설치했던 관부.'}

In [193]:
print(context.iloc[48,0],"\n")
extract_keyword(context.iloc[48,0])

어떤 걸 했냐면 먼저 노비안검법이라는 걸 시행합니다. 왜냐면 호족들이 노비를 많이 갖고 있었거든요. 노비는 뭐예요? 



{'노비안검법': '고려 광종 때 호족세력을 누르고 왕권을 강화하기 위해 본래 양인이었다가 노비가 된 사람을 안검하여 방량(放良)하게 한 일종의 노비해방법.',
 '호족': '신라 말 고려 초의 사회변동을 주도적으로 이끈 지방세력.'}

In [194]:
print(context.iloc[13,0],"\n")
extract_keyword(context.iloc[13,0])

그 새로운 세력들이 누구냐면, 향리 출신의 성리학을 수용한 신진사대부들. 그 신진사대부들이 권문세족을 공격하면서 



{'권문세족': '고려후기 정치세력으로 고려 전기의 문벌귀족, 조선의 양반사대부와 비견되는 지배층.권문세가.'}

In [ ]:
c = context.iloc[:,0].tolist()

In [199]:
for i in range(0,len(c)-10,5):
    text = " ".join(c[i:i+5])
    res = extract_keyword(text)
    if res:
        print("자막 :", text)
        print("추천 결과 :",res)
        print()

자막 : 그 무신들. 이때 원나라가 쳐들어와요. 몽골이 쳐들어와요. 결국은 무너지고 그 몽골에 빌붙어서, 원에 빌붙어서 성장한 세력들이 있으니 그들을 우리는 권문세족이라고 이야기합니다. 이 권문세족. 시대의 막바지에 있는 무너져야 하는 세력들이에요. 그럼 이제 또 새로운 세력이 등장하겠죠. 그 새로운 세력들이 누구냐면, 향리 출신의 성리학을 수용한 신진사대부들. 그 신진사대부들이 권문세족을 공격하면서 바로 새로운 나라 조선을 세우게 되더라. 이 시기 구분을 여러분들이 딱 머릿속에 인지하고 계셔야 돼.
추천 결과 : {'권문세족': '고려후기 정치세력으로 고려 전기의 문벌귀족, 조선의 양반사대부와 비견되는 지배층.권문세가.'}

자막 : 호족, 문벌귀족, 무신, 권문세족 그리고 신진사대부 이렇게 이어지는 흐름. 머릿속에 기억해 놓으시기 바랍니다. 됐죠? 이제부터 하나씩 하나씩 호족의 시대부터 살펴보도록 하겠습니다. 호족의 시대 즉, 고려를 열었던 왕들이 되겠습니다. 태조 왕건이라고 제가 설명드렸는데요. 태조 왕건은 여러 정치인들이 나라를 세웠으니까. 신라 말기에 얼마나 혼란스러웠어요. 원종과 애노의 난처럼 중앙에서 지방을 통제하지 못하니까 난리 났잖아요. 이걸 안정화시켜야 합니다. 민생안정이 목표였어요. 그래서 민생안정을 시키기 위해서 뭘 했을까? 세금을 줄여야죠. 제일 중요한 방법은 세금을 줄여 주는 거예요.
추천 결과 : {'문벌귀족': '대대로 내려오는 그 집안의 사회적 신분이나 지위.'}

자막 : 그 왕이 광종이에요. 미칠 광 자. ‘미친 것 아니야?’ 이런 느낌이 들 정도로. 진짜 미칠 광 자는 아닌데. 호족들에 대한 전면적 숙청, 어마어마한 숙청을 했던 왕이 바로 이 광종입니다. 태조 왕건 때 보였던 왕권의 미약함. 이것을 강화시키기 위한 것이 광종의 목표였다는 것이죠. 그러기 위해서는 뭘 해야 돼요? 호족들을 숙청해야죠. 어떤 걸 했냐면 먼저 노비안검법이라는 걸 시행합니다. 왜냐면 호족들이 노비를 많이 갖고 있었거든요. 노비는 뭐예요? 군사력이라는